In [ ]:
!conda create -n bio python=3.9
!conda init bash
!conda config --add channels bioconda

In [ ]:
!conda install -c bioconda fastqc # para analizar la calidad de los reads
!conda install -c bioconda bwa # para alinear los reads contrar un genoma de referencia
!conda install -c bioconda samtools # para transformar el archivo de alieamiento en un formato desplegable de manera gráfica
!conda install -c bioconda igv # para visualizar gráficamente los aineamientos
!conda install -c bioconda bcftools # para identificar las variantes (mutaciones)
!conda install -c conda-forge py-bgzip # para comprimir en DNAref previo al pileup del llamado de variantes

In [ ]:
!mkdir DNAref # para almacenar el DNA de referencia
!mkdir RawReads # para los reads crudos (que vienen del secuenciador)
!mkdir FastQC_rawReads #para almacenar los reportes de FastQC de los datos crudos
!mkdir Trimmed_reads # para almacenar los FASTQ podados
!mkdir FastQC_trimmedReads #para almacenar los reportes de FastQC de los datos podados
!mkdir Aligned_reads # para almacenar los archivos de alineamiento
!mkdir Variant_calling # para almacenar las variantes identificadas

-   Se crearon los directorios y posteriormente se obtuvo los archivos subidos a siveduc para el grupo 5, estos fueron añadidos a RawReads siguiendo las instrucciones del laboratorio, igualmente se aplico el comando "ln -s [archivo libreria] [L(num)R(1|2).fq.gz]" a cada libreria descargada.
-   Se Descargo el FASTA y GFF3 del cromosoma 16 de Homo sapiens (GRCh38) y estos se renombraron a GRCh38 y se agregaron a la carpeta DNAref.

### Control de calidad


In [ ]:
!fastqc RawReads/L16R1.fq -o FastQC_rawReads/
!fastqc RawReads/L16R2.fq -o FastQC_rawReads/

In [ ]:
!fastqc RawReads/L17R1.fq -o FastQC_rawReads/
!fastqc RawReads/L17R2.fq -o FastQC_rawReads/

### Trimming

In [ ]:
!trimmomatic PE RawReads/L16R1.fq RawReads/L16R2.fq Trimmed_reads/p_L16R1.fq Trimmed_reads/u_L16R1.fq Trimmed_reads/p_L16R2.fq Trimmed_reads/u_L16R2.fq LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:20
!trimmomatic PE RawReads/L17R1.fq RawReads/L17R2.fq Trimmed_reads/p_L17R1.fq Trimmed_reads/u_L17R1.fq Trimmed_reads/p_L17R2.fq Trimmed_reads/u_L17R2.fq LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:20

In [ ]:
!for i in `ls Trimmed_reads/p*.fq`; do fastqc $i -o FastQC_trimmedReads/; done

### Alineamiento : Indexacion

In [14]:
!time bwa index DNAref/GRCh38.fasta

[bwa_index] Pack FASTA... 7.72 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=497912844, availableWord=47034604
[BWTIncConstructFromPacked] 10 iterations done. 76325452 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 142216380 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 200776364 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 252820604 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 299073676 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 340179500 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 376710428 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 409175164 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 438025868 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 463664428 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 486

#### Alineamiento

In [23]:
!bwa mem DNAref/GRCh38.fasta Trimmed_reads/p_L16R1.fq Trimmed_reads/p_L16R2.fq > Aligned_reads/L16.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 45400 sequences (10000158 bp)...
[M::process] read 21396 sequences (4696651 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 0, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] skip orientation FR as there are not enough pairs
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 45400 reads in 9.794 CPU sec, 9.762 real sec
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 1, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] skip orientation FR as there are not enough pairs
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 21396 reads in 4.582 CPU sec, 4.513 real sec
[main] Version: 0.